In [1]:
words = open('names.txt', 'r').read().splitlines()

In [2]:
char = sorted(list(set(''.join(words))))
stoi = {a:z+1 for z,a in enumerate(char)}
stoi['.'] = 0
itos = {z:a for a,z in stoi.items()}

In [32]:
import torch
def create_dataset(words):
    block_size = 3
    X, Y =[], []
    for w in words:
        # print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '----->', itos[ix])
            context = context[1:] + [ix]
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
li = int(0.8 * len(words))
ri = int(0.9 * len(words))
Xtr, ytr = create_dataset(words[:li])
Xval, yval = create_dataset(words[li:ri])
Xtst, ytst = create_dataset(words[ri:])

In [46]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 50), generator=g)
W1 = torch.randn((250, 300), generator=g)
b1 = torch.randn(300, generator=g)
W2 = torch.randn((300, 27), generator=g)
b2 = torch.randn(27, generator=g)
params = [C, W1, b1, W2, b2]


In [47]:
import torch.nn.functional as F

for p in params:
    p.requires_grad = True

In [48]:
lri = []
lossi = []
for i in range(30000):
    ix = torch.randint(0, Xtr.shape[0], (32,))
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 250 ) @ W1 + b1)
    logits = h @ W2 + b2
    # counts = logits.exp()
    # probs = counts / counts.sum(1, keepdims = True)
    # loss = -probs[torch.arange(nums), Y].log().mean()
    loss = F.cross_entropy(logits, ytr[ix])
    for p in params:
        p.grad = None    
    loss.backward()

    lr = 1
    for p in params:
        p.data += -lr * p.grad

    # lri.append(lr)
    # lossi.append(loss.item())
# print(loss.item())

In [49]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 250) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, ytr)
loss

tensor(12.7789, grad_fn=<NllLossBackward0>)

In [50]:
emb = C[Xval]
h = torch.tanh(emb.view(-1, 250) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, yval)
loss

tensor(12.9059, grad_fn=<NllLossBackward0>)

In [58]:
g = torch.Generator().manual_seed(2147483647 + 50)
block_size = 5
for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples = 1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))
        

derylani.
deryla.
dersla.
derylani.
derylani.
dengelley.
derylani.
dersla.
eris.
derse.
dengellen.
dersla.
eris.
derylyc.
derylani.
dersla.
dengellen.
derylani.
derylani.
dersla.
